# STYLING A DATAFRAME

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Suppress scientific notation
np.set_printoptions(suppress=True)

In [3]:
df = pd.read_parquet('datasets/employee_list.parquet')

In [2]:
def df_styles(
    hover_color="#3957b1", th_align='left', th_font_size='105%',
    th_font_weight='normal', th_border_style='none',
    border_width='1px', border_color='#474747',
    tr_align='center', tr_font_size='110%',
    tr_font_weight='normal', tr_border_style='none',
    td_align='center', td_font_size='100%', td_font_weight='normal',
    td_border_style='none', caption_side='center',
    caption_font_size='150%',  caption_color='#427fd4',
    caption_font_weight='bold'):

    border_colors = {
        'very dark': '#333',
        'dark': '#3d3d3d',
        'medium': '#474747',
        'light': '#a9a9a9',
        'very light': '#d1d1d1',
    }

    if border_color in border_colors:
        border_color = border_colors.get(border_color)

    return [
        dict(selector="th", props=[('font-size', th_font_size),
                                   ('text-align', th_align),
                                   ('font-weight', th_font_weight),
                                   ('border-style', th_border_style),
                                   ('border-width', border_width),
                                   ('border-color', border_color)]),

        dict(selector="tr", props=[('font-size', tr_font_size),
                                   ("text-align", tr_align),
                                   ('font-weight', tr_font_weight),
                                   ('border-style', tr_border_style),
                                   ('border-width', border_width),
                                   ('border-color', border_color)]),

        dict(selector="td", props=[("font-size", td_font_size),
                                   ("text-align", td_align),
                                   ('font-weight', td_font_weight),
                                   ('border-style', td_border_style),
                                   ('border-width', border_width),
                                   ('border-color', border_color)]),

        dict(selector="caption", props=[("caption-side", caption_side),
                                        ("font-size", caption_font_size),
                                        ("color", caption_color),
                                        ("font-weight", caption_font_weight)]),
        dict(selector='tr:hover',
             props=[("background-color", f"{hover_color}")])
    ]

In [23]:
df.head()

,employee_number,name,company,country,dob,age,department,salary,has_parking_space
0,897028,Kenneth Jensen,Wilson and Sons,India,1983-07-03,38,Management,124790,False
1,463979,Sarah Anderson,"Hernandez, Cunningham and Clark",India,1980-08-09,41,Consulting,103122,True
2,388446,Tracie Rollins,"Hernandez, Cunningham and Clark",Cayman Islands,1987-07-29,34,Consulting,119072,False
3,267447,Seth Smith,Spears-Brown,Germany,1969-03-04,52,System Architect,115653,False
4,401300,Katherine Fields,"Hernandez, Cunningham and Clark",Venezuela,1980-01-26,42,Finance,119412,False


### Create a table

In [40]:
tbl = df.groupby('country').agg(
    number_of_employees=pd.NamedAgg(
        column='name', aggfunc='count'),
    earliest_dob=pd.NamedAgg(
        column='dob', aggfunc='first'),
    mean_salary=pd.NamedAgg(
        column='salary', aggfunc='mean'),
    )

tbl['employees_percent_of_total'] = tbl.number_of_employees.transform(lambda x: x/x.sum())

tbl

,number_of_employees,earliest_dob,mean_salary,employees_percent_of_total
country,,,,
Cayman Islands,8,1987-07-29,103430.375000,0.08
Germany,8,1969-03-04,99889.250000,0.08
India,14,1983-07-03,103775.857143,0.14
Israel,10,1987-08-19,108547.800000,0.10
Japan,11,1989-12-18,115533.454545,0.11
Mauritania,13,1977-03-14,112472.307692,0.13
Suriname,13,1982-04-29,100747.307692,0.13
USA,14,1969-06-05,109212.571429,0.14
Venezuela,9,1980-01-26,111969.666667,0.09


### Table styling

#### Example 1

In [85]:
styles = df_styles(
    border_color='very dark', 
    tr_font_size='115%', 
    th_font_size='110%', 
    tr_font_weight='bold', 
    th_font_weight='bold')

format_dict = {
    'earliest_dob':'{:%d/%m/%y}', 
    'mean_salary':'${:,.0f}', 
    'employees_percent_of_total':'{:,.0%}'
}

cm = sns.dark_palette('royalblue', as_cmap=True)

(tbl
 .style
    .set_table_styles(styles)
    .format(format_dict)
    .highlight_min(props='color:white; background: darkred', axis=0, subset=['number_of_employees'])
    .highlight_max(props='color:white; background: royalblue', axis=0, subset=['number_of_employees'])
    .background_gradient(cmap=cm, subset=['employees_percent_of_total'])
    .bar(subset=['mean_salary'], color='royalblue')
    .set_caption('Table with different types of styles')
)

,number_of_employees,earliest_dob,mean_salary,employees_percent_of_total
country,,,,
Cayman Islands,8,29/07/87,"$103,430",8%
Germany,8,04/03/69,"$99,889",8%
India,14,03/07/83,"$103,776",14%
Israel,10,19/08/87,"$108,548",10%
Japan,11,18/12/89,"$115,533",11%
Mauritania,13,14/03/77,"$112,472",13%
Suriname,13,29/04/82,"$100,747",13%
USA,14,05/06/69,"$109,213",14%
Venezuela,9,26/01/80,"$111,970",9%


#### Example 2

In [88]:
tbl.style.applymap(lambda x: 'color:lightgreen;')

,number_of_employees,earliest_dob,mean_salary,employees_percent_of_total
country,,,,
Cayman Islands,8,1987-07-29 00:00:00,103430.375000,0.080000
Germany,8,1969-03-04 00:00:00,99889.250000,0.080000
India,14,1983-07-03 00:00:00,103775.857143,0.140000
Israel,10,1987-08-19 00:00:00,108547.800000,0.100000
Japan,11,1989-12-18 00:00:00,115533.454545,0.110000
Mauritania,13,1977-03-14 00:00:00,112472.307692,0.130000
Suriname,13,1982-04-29 00:00:00,100747.307692,0.130000
USA,14,1969-06-05 00:00:00,109212.571429,0.140000
Venezuela,9,1980-01-26 00:00:00,111969.666667,0.090000
